In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(context='talk', style='ticks',
        color_codes=True, rc={'legend.frameon': False})

%matplotlib inline

In [2]:
import tensorflow as tf
import sentencepiece as spm

print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  1


In [3]:
vocab_size = 8000
max_seq_len = 512

sp = spm.SentencePieceProcessor()
sp.Load('sentencepiece_models/uniparc_10M_8000.model')

def sp_encode(line_tensor):
    encoded_array = np.asarray(
        sp.SampleEncodeAsIds(line_tensor.numpy(), nbest_size=-1, alpha=0.5))

    # If the protein sequence is too long, take a random slice.
    if len(encoded_array) > max_seq_len:
        offset = np.random.randint(
            low=0, high=len(encoded_array) - max_seq_len + 1)
        encoded_array = encoded_array[offset:(offset + max_seq_len)]

    return encoded_array

def sp_encode_tf(line_tensor):
    return tf.py_function(sp_encode, inp=[line_tensor], Tout=[tf.int32,])

In [4]:
from bert.dataset import create_masked_input_dataset

training_data = create_masked_input_dataset(
    encode_fn=sp_encode_tf,
    sequence_path='/projects/bpms/pstjohn/uniparc/sequences_train.txt',
    max_sequence_length=max_seq_len,
    batch_size=10,
    buffer_size=1024,
    vocab_size=vocab_size,
    mask_index=4,
    vocab_start=5,
    fix_sequence_length=True)

training_data.repeat().prefetch(tf.data.experimental.AUTOTUNE)

valid_data = create_masked_input_dataset(
    encode_fn=sp_encode_tf,
    sequence_path='/projects/bpms/pstjohn/uniparc/sequences_valid.txt',
    max_sequence_length=max_seq_len,
    batch_size=10,
    buffer_size=1024,
    vocab_size=vocab_size,
    mask_index=4,
    vocab_start=5,
    fix_sequence_length=True)

valid_data.prefetch(tf.data.experimental.AUTOTUNE)

<PrefetchDataset shapes: (((None, 512), (None, 512)), (None, 512, 1)), types: ((tf.int32, tf.bool), tf.int32)>

In [24]:
(masked_seqs, input_masks), true_values = next(iter(training_data))

In [6]:
masked_seqs.numpy().max()

7982

In [7]:
true_values.shape

TensorShape([10, 512, 1])

## BERT layers development

In [22]:
from tensorflow.keras import layers
from bert.layers import (PositionEmbedding, Attention, Transformer, TokenEmbedding, Bias,
                         Projection, gelu, masked_sparse_cross_entropy_loss, InverseSquareRootSchedule,
                         initializer)

vocab_size = 8000
max_seq_len = 512
embedding_dimension = 32
model_dimension = 64
transformer_ff_dimension = model_dimension * 4
num_attention_heads = model_dimension // 16
num_transformer_layers = 4

dropout_rate = 0.1

# Horovod: adjust learning rate based on number of GPUs.
learning_rate = 1E-4
warmup_updates = 3000


inputs = layers.Input(shape=(max_seq_len,), dtype=tf.int32, batch_size=None)
# input_mask = layers.Input(shape=(max_seq_len,), dtype=tf.bool, batch_size=None)

token_embedding_layer = TokenEmbedding(
    vocab_size, embedding_dimension, embeddings_initializer=initializer(), mask_zero=True)
token_embeddings = token_embedding_layer(inputs)
position_embeddings = PositionEmbedding(
    max_seq_len + 1, embedding_dimension, embeddings_initializer=initializer(),
    mask_zero=True)(inputs)

embeddings = layers.Add()([token_embeddings, position_embeddings])
embeddings = Projection(model_dimension, use_residual=False)(embeddings)

transformer = Transformer(num_attention_heads, transformer_ff_dimension, dropout=dropout_rate)
for i in range(num_transformer_layers):
    embeddings = transformer(embeddings)

out = layers.Dense(embedding_dimension, activation=gelu, kernel_initializer=initializer())(embeddings)
out = token_embedding_layer(out, transpose=True)
# out = Bias()([out, input_mask])
# out = layers.Softmax()(out)

model = tf.keras.Model([inputs], [out], name='model')
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            [(None, 512)]        0                                            
__________________________________________________________________________________________________
token_embedding_3 (TokenEmbeddi multiple             256000      input_7[0][0]                    
                                                                 dense_3[0][0]                    
__________________________________________________________________________________________________
position_embedding_3 (PositionE (None, 512, 32)      16416       input_7[0][0]                    
__________________________________________________________________________________________________
add_3 (Add)                     (None, 512, 32)      0           token_embedding_3[0][0]      

In [31]:
inputs = model([masked_seqs])

In [37]:
out = tf.boolean_mask(inputs, input_masks)

In [38]:
out

<tf.Tensor: id=390444, shape=(183, 8000), dtype=float32, numpy=
array([[-0.012174  , -0.01447492,  0.00452682, ..., -0.0060373 ,
         0.00932042, -0.01318178],
       [-0.0010614 , -0.00480176,  0.01051812, ..., -0.00745522,
        -0.00676224, -0.00630224],
       [ 0.01036556, -0.00018116,  0.00781351, ..., -0.0041775 ,
        -0.01215548, -0.00032609],
       ...,
       [-0.01486421, -0.0134879 , -0.00079376, ..., -0.00024359,
         0.00672316, -0.00470749],
       [ 0.0071736 , -0.00071082,  0.00481169, ..., -0.01370568,
         0.0018601 , -0.00494374],
       [ 0.01894609,  0.00136959,  0.01398921, ..., -0.01621576,
        -0.01224323,  0.00046754]], dtype=float32)>

In [9]:
# import tensorflow_addons as tfa

# Horovod: add Horovod DistributedOptimizer.
opt = tf.optimizers.Adam(learning_rate=learning_rate)

model.compile(
    loss=tf.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy'],
    optimizer=opt)

# model.compile(
#     loss=tf.keras.losses.SparseCategoricalCrossentropy(
#         from_logits=True, reduction=tf.keras.losses.Reduction.NONE),
#     metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
#     optimizer=opt)

In [10]:
model.fit(training_data, steps_per_epoch=100, epochs=3, verbose=1,
          validation_data=valid_data, validation_steps=10)

Train for 100 steps, validate for 10 steps
Epoch 1/3


/home/pstjohn/miniconda3/envs/tf2/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/home/pstjohn/miniconda3/envs/tf2/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


100/100 [==============================] - 16s 159ms/step - loss: 0.4173 - accuracy: 0.0171 - val_loss: 0.3895 - val_accuracy: 0.0147
Epoch 2/3
100/100 [==============================] - 7s 66ms/step - loss: 0.3968 - accuracy: 0.0213 - val_loss: 0.3954 - val_accuracy: 0.0181
Epoch 3/3
100/100 [==============================] - 7s 66ms/step - loss: 0.3692 - accuracy: 0.0210 - val_loss: 0.3502 - val_accuracy: 0.0177


In [11]:
model.save('test_model.h5')

In [13]:
model = tf.keras.models.load_model(
    'test_model.h5',
    custom_objects={
        'PositionEmbedding': PositionEmbedding,
        'TokenEmbedding': TokenEmbedding,
        'Attention': Attention,
        'Transformer': Transformer,
        'Projection': Projection,        
        'Bias': Bias,
        'gelu': gelu,
    })

# true_labels = layers.Input(shape=(None,), dtype=tf.int32, batch_size=None)
# model.compile(loss=masked_sparse_cross_entropy_loss, target_tensors=true_labels,
#               optimizer=tfa.optimizers.AdamW(weight_decay=0.01, learning_rate=1E-3))

/home/pstjohn/miniconda3/envs/tf2/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [ ]:
model.fit(training_data, steps_per_epoch=100, epochs=3, verbose=1,
          validation_data=valid_data, validation_steps=10)

In [ ]:
# eval_data = valid_data.map(sp_encode_tf, num_parallel_calls=tf.data.experimental.AUTOTUNE).padded_batch(60, padded_shapes=([512],))
# eval_encoded = next(iter(eval_data))

In [ ]:
bert_predict = model.predict(encoded_data.take(3), verbose=1)

In [ ]:
bert_predict.shape